In [30]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [31]:
import cv2
import numpy as np
from PIL import Image

In [32]:
def data2binary(data):
    if type(data) == str:
        return ''.join([format(ord(i),"08b") for i in data])
    elif type(data) == bytes or type(data) == np.ndarray:
        return [format(i,"08b") for i in data]

In [33]:
def hideData(image,secret_data):
    secret_data += "#####"

    data_index = 0
    binary_data = data2binary(secret_data)
    data_length = len(binary_data)

    for values in image:
        for pixel in values:

            r,g,b = data2binary(pixel)

            if data_index < data_length:
                pixel[0] = int(r[:-1] + binary_data[data_index])
                data_index += 1
            if data_index < data_length:
                pixel[1] = int(g[:-1] + binary_data[data_index])
                data_index += 1
            if data_index < data_length:
                pixel[2] = int(b[:-1] + binary_data[data_index])
                data_index += 1
            if data_index >= data_length:
                break

    return image

In [34]:
def encode_text():
    # image_name = input("Enter Cover Image Name : ")
    image = cv2.imread('/content/drive/MyDrive/Thesis_Final/Thesis_Final/project3/nature.png')
    # image = Image.open("/content/drive/MyDrive/Thesis_Final/Thesis_Final/project3/nature.png")

    data = input("Enter The Text You Want To Hide : ")
    if data == 0:
        raise ValueError("Data is Empty ... ")

    file_name = input("Enter The Encoded Image Name : ")

    encoded_data = hideData(image,data)
    cv2.imwrite(file_name,encoded_data)

In [35]:
def show_data(image):
    binary_data = ""
    for values in image:
        for pixel in values:
            r,g,b = data2binary(pixel)

            binary_data += r[-1]
            binary_data += g[-1]
            binary_data += b[-1]

    all_bytes = [binary_data[i: i+8] for i in range (0,len(binary_data),8)]

    decoded_data = ""
    for byte in all_bytes:
        decoded_data += chr(int(byte,2))
        if decoded_data[-5:] == "#####":
            break

    return decoded_data[:-5]

In [36]:
def decode_text():
    image_name = input("Enter Image You Want To Extract : ")
    image = cv2.imread(image_name)

    text=show_data(image)
    return text

In [37]:
def hideData(image, secret_data):
    secret_data += "#####"
    binary_data = data2binary(secret_data)
    data_len = len(binary_data)
    data_index = 0

    for row in image:
        for pixel in row:
            for i in range(3):  # R, G, B
                if data_index < data_len:
                    pixel[i] = int(format(pixel[i], '08b')[:-1] + binary_data[data_index], 2)
                    data_index += 1
            if data_index >= data_len:
                break
        if data_index >= data_len:
            break
    return image


In [38]:
def show_data(image):
    binary_data = ""
    stop_marker = "#####"
    stop_marker_binary = data2binary(stop_marker)

    for row in image:
        for pixel in row:
            for i in range(3):  # R, G, B
                binary_data += format(pixel[i], '08b')[-1]

                # Check after every 8 bits (1 char)
                if len(binary_data) >= len(stop_marker_binary):
                    # Check last N bits to see if we hit the end marker
                    segment = binary_data[-len(stop_marker_binary):]
                    if segment == stop_marker_binary:
                        # Full message found, break all loops
                        all_bytes = [binary_data[i:i+8] for i in range(0, len(binary_data) - len(stop_marker_binary), 8)]
                        decoded_data = ''.join([chr(int(byte, 2)) for byte in all_bytes])
                        return decoded_data

    # If no end marker is found
    print("⚠️ Warning: End marker not found. Image might not contain hidden data.")
    return ""


In [40]:
def stegnography():
    userinput = int(input("\nSTEGBYTE\n\n 1. Encode \n 2. Decode \n\n Enter Here : "))
    if userinput == 1:
      encode_text()
    else:
        final_data=decode_text()
        print("\nDecoded Data : ",final_data)

stegnography()


STEGBYTE

 1. Encode 
 2. Decode 

 Enter Here : 2
Enter Image You Want To Extract : nature.png

Decoded Data :  go home
